# 0 - Setup

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:.3f}')
pd.set_option('display.max_colwidth', 100)

import numpy as np 
from matplotlib import pyplot as plt

%pip install -r requirements.txt
from google.cloud import bigquery
pmc = "`lemoncake-prod.serving_layer.modelo_para_calculos_finais_controle_recebimentos_join_cobrancas_dynamo`"
project_id = 'lemoncake-prod'
client = bigquery.Client(project = project_id)

%pip install plotly==5.17.0
import plotly.graph_objects as go
import plotly.express as px

from datetime import datetime, timedelta

ERROR: Ignored the following versions that require a different python version: 2.10.0 Requires-Python >=3.6, <3.10; 2.11.0 Requires-Python >=3.6, <3.10; 2.12.0 Requires-Python >=3.6, <3.10; 2.13.0 Requires-Python >=3.6, <3.10; 2.13.1 Requires-Python >=3.6, <3.10; 2.14.0 Requires-Python >=3.6, <3.10; 2.15.0 Requires-Python >=3.6, <3.10; 2.16.0 Requires-Python >=3.6, <3.10; 2.16.1 Requires-Python >=3.6, <3.10; 2.17.0 Requires-Python >=3.6, <3.10; 2.18.0 Requires-Python >=3.6, <3.10; 2.19.0 Requires-Python >=3.6, <3.10; 2.20.0 Requires-Python >=3.6, <3.10; 2.21.0 Requires-Python >=3.6, <3.10; 2.22.0 Requires-Python >=3.6, <3.10; 2.22.1 Requires-Python >=3.6, <3.10; 2.23.0 Requires-Python >=3.6, <3.10; 2.23.1 Requires-Python >=3.6, <3.10; 2.23.2 Requires-Python >=3.6, <3.10; 2.23.3 Requires-Python >=3.6, <3.10; 2.24.0 Requires-Python >=3.6, <3.10; 2.24.1 Requires-Python >=3.6, <3.10; 2.25.0 Requires-Python >=3.6, <3.10; 2.25.1 Requires-Python >=3.6, <3.10; 2.25.2 Requires-Python >=3.6, <3.

Note: you may need to restart the kernel to use updated packages.


c:\Users\DELL\anaconda3\Lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Note: you may need to restart the kernel to use updated packages.


# 1 - Lendo os Dados

In [10]:
data_inicio = "'2020-01-01'"
data_fim = f"'{datetime.strftime(datetime.now(), '%Y-%m-%d')}'"
verde_lemon = 'rgb(0,128,89)'
disco_n_desejadas = "('CELPE')"
n_min_cobrancas = 1

clusters_colors = {1:'rgb(5,191,255)',
                   2:'rgb(5,191,217)',
                   3:'rgb(157,181,123)',
                   4:'rgb(200,50,57)',
                   5:'rgb(255,26,49)',
                   6:'rgb(83,0,116)',
                   }

In [16]:
data_fim

"'2023-12-20'"

In [17]:
data_inicio

"'2020-01-01'"

In [18]:
query = f'''WITH boletos_de_interesse AS(
            SELECT
                --CONCAT(IFNULL(TRIM(UPPER(razao_social)),''),'_',IFNULL(TRIM(UPPER(documento_customer)),'')) AS cliente,
                IFNULL(TRIM(UPPER(razao_social)),'') AS cliente,
                IFNULL(UPPER(documento_customer),'') AS documento_customer,
                nr_instalacao,
                disco, 
                gerador, 
                chave,  
                data_emissao, 
                data_vencimento,
                data_liquidacao,
                valor_original_brl,  
            CASE 
                WHEN data_liquidacao IS NULL THEN 'em_aberto'
                ELSE 'pago'
            END AS situacao_de_pgto,
            CASE 
                WHEN data_liquidacao IS NULL THEN valor_original_brl
                ELSE 0
            END as valor_em_aberto,
            CASE
                WHEN data_liquidacao > data_vencimento THEN valor_original_brl
                WHEN (data_liquidacao IS NULL) AND (data_vencimento < {data_fim}) THEN valor_original_brl
                ELSE 0
            END AS valor_divida,
            CASE 
                WHEN data_liquidacao IS NULL THEN EXTRACT(DAY FROM {data_fim} - data_vencimento)
                ELSE EXTRACT(DAY FROM data_liquidacao - data_vencimento)
            END AS dias_sem_pgto, 
            FROM {pmc}
                WHERE data_emissao BETWEEN {data_inicio} AND {data_fim}
                AND data_vencimento <= {data_fim}
                AND chave != 'Original Parcelado'
                AND disco NOT IN {disco_n_desejadas}
                ORDER BY cliente DESC, data_emissao ASC
            )
            SELECT
                cliente,
                ARRAY_AGG(documento_customer) AS documento_customer,
                COUNT(*) AS qt_de_cobrancas,
                ARRAY_AGG(nr_instalacao) AS nrs_de_instalacao,
                ARRAY_AGG(disco) AS discos,
                ARRAY_AGG(gerador) AS geradores, 
                ARRAY_AGG(chave) AS chaves,
                ARRAY_AGG(data_emissao) AS datas_de_emissao, 
                ARRAY_AGG(data_vencimento) AS datas_de_vencimento, 
                ARRAY_AGG(CASE WHEN data_liquidacao IS NULL THEN 'sem_data_de_liquidacao' ELSE CAST(data_liquidacao AS STRING) END) AS datas_de_liquidacao,
                ARRAY_AGG(valor_original_brl) AS valores_originais,
                ARRAY_AGG(valor_em_aberto) AS valores_em_aberto,
                ARRAY_AGG(valor_divida) AS valores_divida, 
                ARRAY_AGG(situacao_de_pgto) AS situacoes_de_pagamento,
                ARRAY_AGG(dias_sem_pgto) AS dias_sem_pgto, 
                ARRAY_AGG(CASE WHEN dias_sem_pgto > 0 THEN FALSE ELSE TRUE END) as pgto_em_dia
                FROM boletos_de_interesse
            GROUP BY cliente'''

In [19]:
dados_clientes_raw = client.query(query=query).to_dataframe()

cond = dados_clientes_raw['qt_de_cobrancas'] >= n_min_cobrancas
dados_clientes_raw = dados_clientes_raw.loc[cond,:].copy()

In [21]:
def get_main_disco(x):
    not_null_discos = [d for d in x if d!= '']
    if len(set(not_null_discos))>1:
        values,counts = np.unique(x,return_counts=True)
        counts_dict = dict(zip(counts,values))
        most_freq = counts_dict[max(list(counts_dict.keys()))]
        result = most_freq
    elif len(set(not_null_discos)) == 1:
        result = not_null_discos[0]        
    return result

def get_main_gerador(x):
    not_null_geradores = [g for g in x if g != '']
    if len(set(not_null_geradores))>1:
        values,counts = np.unique(x,return_counts=True)
        counts_dict = dict(zip(counts,values))
        most_freq = counts_dict[max(list(counts_dict.keys()))]
        result = most_freq
    elif len(set(not_null_geradores)) == 1:
        result = not_null_geradores[0]        
    return result

def get_n_de_dividas(x):
    values,counts = np.unique(x,return_counts=True)
    counts_dict = dict(zip(values,counts))
    if False in set(counts_dict.keys()):
        n_dividas = counts_dict[False]
    else:           
        n_dividas = 0
    return n_dividas

def get_n_dividas_em_aberto(x):
    values,counts = np.unique(x,return_counts=True)
    counts_dict = dict(zip(values,counts))
    if 'em_aberto' in set(counts_dict.keys()):
        n_dividas_em_aberto = counts_dict['em_aberto']
    else:           
        n_dividas_em_aberto = 0
    return n_dividas_em_aberto

def get_ultima_data_emissao(x):
    return x.max()

def get_ticket_medio(x):
    return x.mean()

dados_clientes_raw['disco_principal'] = dados_clientes_raw['discos'].apply(get_main_disco)
dados_clientes_raw['gerador_principal'] = dados_clientes_raw['geradores'].apply(get_main_gerador)
dados_clientes_raw['ultima_data_de_emissao'] = dados_clientes_raw['datas_de_emissao'].apply(get_ultima_data_emissao)
dados_clientes_raw['ticket_medio'] = dados_clientes_raw['valores_originais'].apply(get_ticket_medio)
dados_clientes_raw['n_total_de_dividas'] = dados_clientes_raw['pgto_em_dia'].apply(get_n_de_dividas)
dados_clientes_raw['n_dividas_em_aberto'] = dados_clientes_raw['situacoes_de_pagamento'].apply(get_n_dividas_em_aberto)

In [35]:
dados_clientes_raw['percentual_cobrancas_pagas'] = 1 - (dados_clientes_raw['n_dividas_em_aberto']/dados_clientes_raw['qt_de_cobrancas'])

In [22]:
def get_frequencia(x):
    return (~x).sum()/len(x)

dados_clientes_raw['frequencia'] = dados_clientes_raw['pgto_em_dia'].apply(get_frequencia)

In [23]:
def get_severidade(x):
    return x.mean()

dados_clientes_raw['severidade']  = dados_clientes_raw['dias_sem_pgto'].apply(get_severidade)

In [24]:
def get_divida_media(x):
    return x.mean()

dados_clientes_raw['divida_media'] = dados_clientes_raw['valores_divida'].apply(get_divida_media)

In [25]:
def get_total_em_aberto(x):
    return x.sum()

dados_clientes_raw['valor_total_em_aberto'] = dados_clientes_raw['valores_em_aberto'].apply(get_total_em_aberto)

# 2 - Calculando Bucktes de Atraso 

In [122]:
cols_interesse = ['cliente',
                  'frequencia',
                  'severidade',
                  'percentual_cobrancas_pagas',
                  'n_dividas_em_aberto',
                  'dias_sem_pgto',
                  'datas_de_emissao',
                  'datas_de_vencimento',
                  'datas_de_liquidacao',
                  'valores_originais',
                  'situacoes_de_pagamento']

In [123]:
def get_faixas_de_atraso(df,perc_pag_min, ultima_data_emissao_min):
    cond0 = df['percentual_cobrancas_pagas'] >= perc_pag_min
    cond1 = df['datas_de_emissao'].apply(lambda x: x[-1]) >= ultima_data_emissao_min
    cond = cond0 & cond1

    clientes_interesse = df.loc[cond,cols_interesse].copy()

    limites = [-float('inf'),0,10,20,30,40,50,60,float('inf')]
    rotulos = ['menor_ou_igual_0',
            'entre 0 e 10',
            'entre 10 e 20', 
            'entre 20 e 30', 
            'entre 30 e 40',
            'entre 40 e 50',
            'entre 50 e 60',
            'maior do que 60']

    clientes_interesse['faixa_de_atraso_medio'] = pd.cut(clientes_interesse['severidade'], bins = limites, labels=rotulos, right = False)
    result = clientes_interesse['faixa_de_atraso_medio'].value_counts(normalize=True, sort = False).to_frame()
    result.reset_index(drop=False, inplace=True)
    result.rename(columns={'index':'faixa_de_atraso',
                        'faixa_de_atraso_medio':'percentual_de_clientes'},
                         inplace = True)
    
    result_qt = clientes_interesse['faixa_de_atraso_medio'].value_counts(normalize=False, sort = False).to_frame()
    result_qt.reset_index(drop=False, inplace=True)
    result_qt.rename(columns={'index':'faixa_de_atraso',
                        'faixa_de_atraso_medio':'quantidade_de_clientes'},
                         inplace = True)
    
    result_final = pd.merge(right=result_qt,
                            left = result,
                            on = 'faixa_de_atraso',
                            how = 'inner')
    
    clientes_interesse.sort_values(by='severidade', ascending=True, inplace=True)

    
    
    return result_final, clientes_interesse

In [124]:
faixas, clientes = get_faixas_de_atraso(df = dados_clientes_raw[cols_interesse],
                                        perc_pag_min = 0.9,
                                        ultima_data_emissao_min = '2023-01-01')
faixas                   

,faixa_de_atraso,percentual_de_clientes,quantidade_de_clientes
0,menor_ou_igual_0,0.319,1243
1,entre 0 e 10,0.536,2088
2,entre 10 e 20,0.091,355
3,entre 20 e 30,0.040,156
4,entre 30 e 40,0.009,37
5,entre 40 e 50,0.003,10
6,entre 50 e 60,0.001,2
7,maior do que 60,0.001,4


In [125]:
faixas.to_csv('faixas_de_atraso.csv', index=False)
clientes.to_csv('clientes.csv', index = False)